Our target is to get the degree 2 polynomials $H_i^L, H_i^C, H_i^R$ using $(i-2,i-1,i),(i-1,i,i+1), (i,i+1,i+2)$ respectively.

So, we create a function that generates symbolic polynomial like that. Basically, if we give $\{-1,0,1\}$ as the point, we would be creating a polynomial $H$ such that 
$$
\int_{-3/2}^{-1/2} H = f_{i-1}, \quad \int_{-1/2}^{1/2} H = f_{i}, \quad  \int_{1/2}^{3/2} H = f_{i+1}
$$
where $f_j$'s would also be symbolic. In general, a degree $k$ approximation would require inputs $\{i-j-k,\dots,i-j,\dots,i-j+k\}$. Then, we would enforce the condition, for each $l$ in these indices, 
$$
\int_{l-1/2}^{l+1/2} H = f_l
$$

We'd use the following sympy resources 

https://docs.sympy.org/latest/modules/integrals/integrals.html

https://medium.com/swlh/basics-of-solving-equations-in-sympy-ad770c14debd

https://stackoverflow.com/questions/25985338/solving-a-system-of-algebraic-equations-with-sympy

First, let's see whether we can successfully generate one symbolic equation out of 
$$
\int_{l-1/2}^{l+1/2} H = f_l.
$$

In [21]:
from sympy import *
l = 2
init_printing(use_unicode=False, wrap_line=False)
xi  = Symbol('xi')
al = Symbol('a'+str(l))
integrand0 = al
bl = Symbol('b'+str(l))
bl = bl*xi
integrand0 += bl
cl = Symbol('c'+str(l))
cl = cl*(xi**2-1.0/12.0)
integrand0 += cl
integrate(integrand0,(xi,(l-1/2,l+1/2)))

The way we have written it, it appears that we have the freedom to write both the degree of approximation and the points in the function. However, the names of symbols `a,b,c` isn't good. Let's try to replace it with something else.

In [45]:
from sympy import *
l = 0
init_printing(use_unicode=False, wrap_line=False)
xi  = Symbol('xi')
sym = Symbol('a_{'+str(0)+','+str(l)+'}')
integrand0 = sym
sym = Symbol('a_{'+str(1)+','+str(l)+'}')
sym = sym*xi
integrand0 += sym
sym = Symbol('a_{'+str(2)+','+str(l)+'}')
sym = sym*(xi**2-1.0/12.0)
integrand0 += sym
integrate(integrand0,(xi,(l-1/2,l+1/2)))

Yes, this is exactly the abstraction we needed!
Now, we create a function that generates for degree $k$, the condition centred at $l$

In [50]:
def gen_condn(degree, centre):
    k = degree
    l = centre
    xi  = Symbol('xi')
    init_printing(use_unicode=False, wrap_line=False)
    integrand = 0*xi
    for i in range(0,k+1):
        sym = Symbol('a_{'+str(i)+','+str(l)+'}')
        sym = sym*xi**i
        integrand += sym
    lhs_condition = integrate(integrand,(xi,(l-1/2,l+1/2)))
    return integrand,lhs_condition

In [71]:
gen_condn(2,0)

Okay, we have successfuly built an algorithm that generates polynomials with symbolic coefficients in monomials. But, we want to do it in Legendre basis. So, we write them explicity.

In [64]:
def leg(degree,xi):
    n = degree
    if n==0:
        return 1.0 + xi*0.0
    if n==1:
        return xi
    if n==2:
        return xi**2-1.0/12.0
    if n==3:
        return xi**3 - (3.0/20.0)*xi
    if n==4:
        return xi**4 - (3.0/14.0)*xi**2 + (3.0/560.0)

In [66]:
def gen_condn(degree, centre):
    k = degree
    l = centre
    xi  = Symbol('xi')
    init_printing(use_unicode=False, wrap_line=False)
    integrand = 0*xi
    for i in range(0,k+1):
        sym = Symbol('a_{'+str(i)+','+str(l)+'}')
        sym = sym*leg(i,xi)
        integrand += sym
    lhs_condition = integrate(integrand,(xi,(l-1/2,l+1/2)))
    return integrand,lhs_condition

In [72]:
def get_condns(degree,centre):
    k = degree
    l = centre
    return